In [1]:
import numpy as np
import seaborn as sb
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Grouper
from pandas import DataFrame
import csv
import pickle
import math
import matrixprofile as mp
from matplotlib.patches import Rectangle
from fbprophet import Prophet
from prophet.plot import plot_yearly
from sklearn.metrics import mean_squared_error, mean_absolute_error
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from sklearn.model_selection import TimeSeriesSplit
from numpy import sqrt
import datetime
from datetime import timedelta
import plotly.graph_objs as go
from sklearn.metrics import r2_score

# Modelo 0-5

In [2]:
df_final = pd.read_csv("/home/jonathan/tesis/11avo_avance/SerieTempFeb_Jul.csv", sep=',',index_col=0, parse_dates=True)    
df_final = df_final.rename(columns={df_final.columns[0]:'y'})
df_final.head()

,y
ds,
2020-02-14 00:00:00,308.104
2020-02-14 01:00:00,165.496
2020-02-14 02:00:00,155.760
2020-02-14 03:00:00,126.662
2020-02-14 04:00:00,123.672


In [3]:
# Agregamos el lunes anterior
for i in range(168,0,-168):
    df_final['t-'+str(i)] = df_final["y"].shift(i)
    df_final = df_final.fillna(0)

In [4]:
Acc_ds1 = pd.read_csv("/home/jonathan/tesis/11avo_avance/Acc_ds1.csv",index_col=0, parse_dates=True)
Acc_ds2 = pd.read_csv("/home/jonathan/tesis/11avo_avance/Acc_ds2.csv",index_col=0, parse_dates=True)

df_final['Accidente1'] = Acc_ds1.Accidente
df_final['Accidente2'] = Acc_ds2.Accidente
df_final.Accidente1.fillna(False,inplace=True)
df_final.Accidente2.fillna(False,inplace=True)

df_final = df_final.reset_index()

In [5]:
def est_Lunes0_5(ds):
    global fila    
    global valor 
    date = pd.to_datetime(ds)
    if (date.dayofweek == 0 and date.hour == 0) or (date.dayofweek == 0 and date.hour == 1) or (date.dayofweek == 0 and date.hour == 2) or (date.dayofweek == 0 and date.hour == 3) or (date.dayofweek == 0 and date.hour == 4) or (date.dayofweek == 0 and date.hour == 5):
        Acc1 = df_final.loc[fila]   
        Acc1 = Acc1['Accidente1']
        Acc2 = df_final.loc[fila]   
        Acc2 = Acc2['Accidente2']
        if (Acc1 == True) or (Acc2 == True):
            valor = True
        else:
            valor = True        
        fila += 1
    else:
        valor = False        
        fila += 1
    return (valor)

def est_Lunes6_11(ds):
    global fila    
    global valor 
    date = pd.to_datetime(ds)
    if (date.dayofweek == 0 and date.hour == 6) or (date.dayofweek == 0 and date.hour == 7) or (date.dayofweek == 0 and date.hour == 8) or (date.dayofweek == 0 and date.hour == 9) or (date.dayofweek == 0 and date.hour == 10) or (date.dayofweek == 0 and date.hour == 11):
        Acc1 = df_final.loc[fila]   
        Acc1 = Acc1['Accidente1']
        Acc2 = df_final.loc[fila]   
        Acc2 = Acc2['Accidente2']
        if (Acc1 == True) or (Acc2 == True):
            valor = True
        else:
            valor = True        
        fila += 1
    else:
        valor = False        
        fila += 1
    return (valor)

def est_Lunes12_17(ds):
    global fila    
    global valor 
    date = pd.to_datetime(ds)
    if (date.dayofweek == 0 and date.hour == 12) or (date.dayofweek == 0 and date.hour == 13) or (date.dayofweek == 0 and date.hour == 14) or (date.dayofweek == 0 and date.hour == 15) or (date.dayofweek == 0 and date.hour == 16) or (date.dayofweek == 0 and date.hour == 17):
        Acc1 = df_final.loc[fila]   
        Acc1 = Acc1['Accidente1']
        Acc2 = df_final.loc[fila]   
        Acc2 = Acc2['Accidente2']
        if (Acc1 == True) or (Acc2 == True):
            valor = True
        else:
            valor = True        
        fila += 1
    else:
        valor = False        
        fila += 1
    return (valor)

def Regressor_YMod(ds):
    global fila    
    global valor     
    date = pd.to_datetime(ds)
    consulta = df_final.loc[fila]
    Acc1 = consulta['Accidente1']
    Acc2 = consulta['Accidente2']
    if (Acc1 == True) & (Acc2 == False):        
        valor = consulta['y']*1.1
    elif (Acc1 == False) & (Acc2 == True):        
        valor = consulta['y']*0.7
    elif (Acc1 == True) & (Acc2 == True):        
        valor = consulta['y']
    elif (Acc1 == False) & (Acc2 == False):        
        valor = consulta['y']
    else:        
        valor = consulta['y']
    fila += 1
    return(valor)

fila = 0
df_final['Lunes0_5'] = df_final['ds'].apply(est_Lunes0_5)
fila = 0
df_final['Lunes6_11'] = df_final['ds'].apply(est_Lunes6_11)
fila = 0
df_final['Lunes12_17'] = df_final['ds'].apply(est_Lunes12_17)
fila = 0
df_final['Y_Mod'] = df_final['ds'].apply(Regressor_YMod)


In [6]:
for i in range(168,0,-168):
    df_final['t-'+str(i)+'Mod'] = df_final["Y_Mod"].shift(i)
    df_final = df_final.fillna(0)
    
df_final.head()

,ds,y,t-168,Accidente1,Accidente2,Lunes0_5,Lunes6_11,Lunes12_17,Y_Mod,t-168Mod
0,2020-02-14 00:00:00,308.104,0.0,True,False,False,False,False,338.9144,0.0
1,2020-02-14 01:00:00,165.496,0.0,True,True,False,False,False,165.4960,0.0
2,2020-02-14 02:00:00,155.760,0.0,False,True,False,False,False,109.0320,0.0
3,2020-02-14 03:00:00,126.662,0.0,False,False,False,False,False,126.6620,0.0
4,2020-02-14 04:00:00,123.672,0.0,True,False,False,False,False,136.0392,0.0


In [7]:
df_final.iloc[3943-168:3943-168+20]

,ds,y,t-168,Accidente1,Accidente2,Lunes0_5,Lunes6_11,Lunes12_17,Y_Mod,t-168Mod
3775,2020-07-20 07:00:00,368.023,411.824,True,False,False,True,False,404.8253,411.8240
3776,2020-07-20 08:00:00,545.494,535.627,False,True,False,True,False,381.8458,535.6270
3777,2020-07-20 09:00:00,467.381,477.515,False,False,False,True,False,467.3810,477.5150
3778,2020-07-20 10:00:00,458.262,524.717,True,False,False,True,False,504.0882,524.7170
3779,2020-07-20 11:00:00,508.913,463.746,False,True,False,True,False,356.2391,463.7460
3780,2020-07-20 12:00:00,508.930,543.916,False,False,False,False,True,508.9300,543.9160
3781,2020-07-20 13:00:00,561.969,616.345,False,False,False,False,True,561.9690,616.3450
3782,2020-07-20 14:00:00,539.662,660.434,True,False,False,False,True,593.6282,726.4774
3783,2020-07-20 15:00:00,588.046,698.047,True,True,False,False,True,588.0460,698.0470
3784,2020-07-20 16:00:00,648.325,611.655,False,True,False,False,True,453.8275,428.1585


In [8]:
df_final0_5 = df_final[['ds','y','Lunes0_5','t-168Mod']]
df_final0_5

,ds,y,Lunes0_5,t-168Mod
0,2020-02-14 00:00:00,308.104,False,0.0000
1,2020-02-14 01:00:00,165.496,False,0.0000
2,2020-02-14 02:00:00,155.760,False,0.0000
3,2020-02-14 03:00:00,126.662,False,0.0000
4,2020-02-14 04:00:00,123.672,False,0.0000
...,...,...,...,...
4051,2020-07-31 19:00:00,505.915,False,483.7714
4052,2020-07-31 20:00:00,549.821,False,774.2273
4053,2020-07-31 21:00:00,455.421,False,352.1119
4054,2020-07-31 22:00:00,294.034,False,523.9080


In [9]:
df_final6_11 = df_final[['ds','y','Lunes6_11','t-168Mod']]
df_final6_11

,ds,y,Lunes6_11,t-168Mod
0,2020-02-14 00:00:00,308.104,False,0.0000
1,2020-02-14 01:00:00,165.496,False,0.0000
2,2020-02-14 02:00:00,155.760,False,0.0000
3,2020-02-14 03:00:00,126.662,False,0.0000
4,2020-02-14 04:00:00,123.672,False,0.0000
...,...,...,...,...
4051,2020-07-31 19:00:00,505.915,False,483.7714
4052,2020-07-31 20:00:00,549.821,False,774.2273
4053,2020-07-31 21:00:00,455.421,False,352.1119
4054,2020-07-31 22:00:00,294.034,False,523.9080


In [10]:
df_final12_17 = df_final[['ds','y','Lunes12_17','t-168Mod']]
df_final12_17

,ds,y,Lunes12_17,t-168Mod
0,2020-02-14 00:00:00,308.104,False,0.0000
1,2020-02-14 01:00:00,165.496,False,0.0000
2,2020-02-14 02:00:00,155.760,False,0.0000
3,2020-02-14 03:00:00,126.662,False,0.0000
4,2020-02-14 04:00:00,123.672,False,0.0000
...,...,...,...,...
4051,2020-07-31 19:00:00,505.915,False,483.7714
4052,2020-07-31 20:00:00,549.821,False,774.2273
4053,2020-07-31 21:00:00,455.421,False,352.1119
4054,2020-07-31 22:00:00,294.034,False,523.9080


In [11]:
df_final18_23 = df_final[['ds','y','t-168Mod']]
df_final18_23

,ds,y,t-168Mod
0,2020-02-14 00:00:00,308.104,0.0000
1,2020-02-14 01:00:00,165.496,0.0000
2,2020-02-14 02:00:00,155.760,0.0000
3,2020-02-14 03:00:00,126.662,0.0000
4,2020-02-14 04:00:00,123.672,0.0000
...,...,...,...
4051,2020-07-31 19:00:00,505.915,483.7714
4052,2020-07-31 20:00:00,549.821,774.2273
4053,2020-07-31 21:00:00,455.421,352.1119
4054,2020-07-31 22:00:00,294.034,523.9080


In [12]:
# df_final0_5.to_csv('df_final0_5.csv', index=False)
# df_final6_11.to_csv('df_final6_11.csv', index=False)
# df_final12_17.to_csv('df_final12_17.csv', index=False)
# df_final18_23.to_csv('df_final18_23.csv', index=False)